# Projeto 13: Rede Neural Convolucional MNIST Aumento de Imagens

## Etapa 1: Importação das bibliotecas

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets
import torchvision.transforms as transforms
torch.__version__

'1.4.0'

## Etapa 2: Base de dados

In [ ]:
torch.manual_seed(123)

In [ ]:
transform_train = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(degrees = 7, 
                                                              translate = (0, 0.07), # horizontal and vertical shifts
                                                              shear = 7,
                                                              scale = (1, 1.2) # zoom range
                                                              ),
                                      transforms.ToTensor()
                                      ])

In [ ]:
transform_test = transforms.ToTensor()

In [ ]:
train = datasets.MNIST(root = '.', train = True, download = True, transform = transform_train)
test = datasets.MNIST(root = '.', train = False, download = True, transform = transform_test)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)
test_loader = torch.utils.data.DataLoader(test, batch_size = 128)

## Etapa 3: Construção do modelo

In [ ]:
class classificador(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = (3, 3))
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = (2, 2))
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(in_features = 32*13*13, out_features = 128)
        self.output = nn.Linear(128, 10)
        
    def forward(self, X):
        X = self.pool(self.activation(self.conv(X)))
        X = self.flatten(X)
        X = self.activation(self.linear(X))
        X = self.output(X)

        return X

In [ ]:
net = classificador()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
net.to(device)

classificador(
  (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten()
  (linear): Linear(in_features=5408, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

## Etapa 4: Treinamento do modelo

In [ ]:
def training_loop(loader, epoch):

    running_loss = 0.
    running_accuracy = 0.
    
    for i, data in enumerate(loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()        

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        
        ps = F.softmax(outputs)
        top_p, top_class = ps.topk(k = 1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.float))
        
        running_accuracy += accuracy
                
        # Imprimindo os dados referentes a este loop
        print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:03.2f} - precisão {:03.2f}'.format(epoch + 1, i + 1, len(loader), loss, accuracy), end = '\r')
        
    # Imprimindo os dados referentes a esta época
    
    print('\rÉPOCA {:3d} FINALIZADA: perda {:.5f} - precisão {:.5f}'.format(epoch+1, running_loss/len(loader), running_accuracy/len(loader)))

In [ ]:
for epoch in range(5):
    print('Treinando...')
    training_loop(train_loader, epoch)

    net.eval()
    print('Validando...')
    training_loop(test_loader, epoch)

    net.train()

Treinando...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


ÉPOCA   1 FINALIZADA: perda 0.62339 - precisão 0.80064
Validando...
ÉPOCA   1 FINALIZADA: perda 0.18613 - precisão 0.94422
Treinando...
ÉPOCA   2 FINALIZADA: perda 0.27026 - precisão 0.91640
Validando...
ÉPOCA   2 FINALIZADA: perda 0.10888 - precisão 0.96578
Treinando...
ÉPOCA   3 FINALIZADA: perda 0.19461 - precisão 0.93990
Validando...
ÉPOCA   3 FINALIZADA: perda 0.07838 - precisão 0.97528
Treinando...
ÉPOCA   4 FINALIZADA: perda 0.16171 - precisão 0.95074
Validando...
ÉPOCA   4 FINALIZADA: perda 0.06326 - precisão 0.97913
Treinando...
ÉPOCA   5 FINALIZADA: perda 0.13963 - precisão 0.95723
Validando...
ÉPOCA   5 FINALIZADA: perda 0.05049 - precisão 0.98467
